In [204]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import pow,exp
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import chi2

In [205]:
df = pd.read_csv('climate_change.csv')

In [206]:
df.head()

,Year,Month,MEI,CO2,CH4,N2O,CFC-11,CFC-12,TSI,Aerosols,Temp
0,1983,5,2.556,345.96,1638.59,303.677,191.324,350.113,1366.1024,0.0863,0.109
1,1983,6,2.167,345.52,1633.71,303.746,192.057,351.848,1366.1208,0.0794,0.118
2,1983,7,1.741,344.15,1633.22,303.795,192.818,353.725,1366.2850,0.0731,0.137
3,1983,8,1.130,342.25,1631.35,303.839,193.602,355.633,1366.4202,0.0673,0.176
4,1983,9,0.428,340.17,1648.40,303.901,194.392,357.465,1366.2335,0.0619,0.149


Problem 1.1 - Creating Our First Model
2.0 points possible (graded)
We are interested in how changes in these variables affect future temperatures, as well as how well these variables explain temperature changes so far. To do this, first read the dataset climate_change.csv into R.

Then, split the data into a training set, consisting of all the observations up to and including 2006, and a testing set consisting of the remaining years (hint: use subset). A training set refers to the data that will be used to build the model (this is the data we give to the lm() function), and a testing set refers to the data we will use to test our predictive ability.

Next, build a linear regression model to predict the dependent variable Temp, using MEI, CO2, CH4, N2O, CFC.11, CFC.12, TSI, and Aerosols as independent variables (Year and Month should NOT be used in the model). Use the training set to build the model.

Enter the model R2 (the "Multiple R-squared" value):

In [207]:
df.columns

Index([u'Year', u'Month', u'MEI', u'CO2', u'CH4', u'N2O', u'CFC-11', u'CFC-12',
       u'TSI', u'Aerosols', u'Temp'],
      dtype='object')

In [208]:
df_train = df[df.Year<=2006]
df_test = df[df.Year>2006]
features = [ u'MEI', u'CO2', u'CH4', u'N2O', u'CFC-11', u'CFC-12',
       u'TSI', u'Aerosols']

In [209]:
clf = LinearRegression()
clf.fit(df_train[features],df_train.Temp)
clf.score(df_train[features],df_train.Temp)

0.7508932770523391

Problem 1.2 - Creating Our First Model
1 point possible (graded)

Which variables are significant in the model? We will consider a variable signficant only if the p-value is below 0.05. (Select all that apply.)


In [210]:
df_lr = pd.DataFrame(zip(features,clf.coef_.tolist()),columns=['Feature','weight'])
df_lr['absolute_weight']=np.abs(df_lr.weight)
df_lr.sort_values(by='absolute_weight')

,Feature,weight,absolute_weight
2,CH4,0.000124,0.000124
5,CFC-12,0.003808,0.003808
1,CO2,0.006457,0.006457
4,CFC-11,-0.006630,0.006630
3,N2O,-0.016528,0.016528
0,MEI,0.064205,0.064205
6,TSI,0.093141,0.093141
7,Aerosols,-1.537613,1.537613


In [211]:
_, p_values = f_regression(df_train[features],df_train.Temp)
print  _
df_pvalues = pd.DataFrame(zip(features,p_values.tolist()),columns=['Feature','weight'])
df_pvalues.sort_values(by='weight',ascending=True)

[   8.64559024  463.59447651  275.9381421   434.2424694    56.22183027
  252.83581202   17.75609183   49.04753697]


,Feature,weight
1,CO2,1.741540e-61
3,N2O,5.078804e-59
2,CH4,1.107285e-43
5,CFC-12,4.400163e-41
4,CFC-11,8.441359e-13
7,Aerosols,1.833417e-11
6,TSI,3.383637e-05
0,MEI,3.550173e-03


Problem 2.1 - Understanding the Model
1 point possible (graded)

Current scientific opinion is that nitrous oxide and CFC-11 are greenhouse gases: gases that are able to trap heat from the sun and contribute to the heating of the Earth. However, the regression coefficients of both the N2O and CFC-11 variables are negative, indicating that increasing atmospheric concentrations of either of these two compounds is associated with lower global temperatures.

Which of the following is the simplest correct explanation for this contradiction?


Climate scientists are wrong that N2O and CFC-11 are greenhouse gases - this regression analysis constitutes part of a disproof.

There is not enough data, so the regression coefficients being estimated are not accurate.

All of the gas concentration variables reflect human development - N2O and CFC.11 are correlated with other variables in the data set.

Problem 2.2 - Understanding the Model
2 points possible (graded)
Compute the correlations between all the variables in the training set. Which of the following independent variables is N2O highly correlated with (absolute correlation greater than 0.7)? Select all that apply.



In [212]:
(df_train[features].corr()[['N2O']]>0.7).sort_values(by='N2O')

,N2O
MEI,False
CFC-11,False
TSI,False
Aerosols,False
CO2,True
CH4,True
N2O,True
CFC-12,True


Which of the following independent variables is CFC.11 highly correlated with? Select all that apply.

In [213]:
(df_train[features].corr()[['CFC-11']]>0.7).sort_values(by='CFC-11')

,CFC-11
MEI,False
CO2,False
N2O,False
TSI,False
Aerosols,False
CH4,True
CFC-11,True
CFC-12,True


Problem 3 - Simplifying the Model
2.0 points possible (graded)

Given that the correlations are so high, let us focus on the N2O variable and build a model with only MEI, TSI, Aerosols and N2O as independent variables. Remember to use the training set to build the model.

Enter the coefficient of N2O in this reduced model:



In [214]:
features_simplified = [ u'MEI', u'N2O',u'TSI', u'Aerosols']

In [215]:
clf = LinearRegression()
clf.fit(df_train[features_simplified],df_train.Temp)
print 'coef N2O: ',clf.coef_[1]
print 'R2 : ',clf.score(df_train[features_simplified],df_train.Temp)


coef N2O:  0.025319745729
R2 :  0.726132127951


Problem 4 - Automatically Building the Model
4.0 points possible (graded)

We have many variables in this problem, and as we have seen above, dropping some from the model does not decrease model quality. R provides a function, step, that will automate the procedure of trying different combinations of variables to find a good compromise of model simplicity and R2. This trade-off is formalized by the Akaike information criterion (AIC) - it can be informally thought of as the quality of the model with a penalty for the number of variables in the model.

The step function has one argument - the name of the initial model. It returns a simplified model. Use the step function in R to derive a new model, with the full model as the initial model (HINT: If your initial full model was called "climateLM", you could create a new model with the step function by typing step(climateLM). Be sure to save your new model to a variable name so that you can look at the summary. For more information about the step function, type ?step in your R console.)

Enter the R2 value of the model produced by the step function:



In [220]:
#simple conversion of step method for R into Python
# source http://stackoverflow.com/questions/22428625/does-statsmodels-or-another-python-package-offer-an-equivalent-to-rs-step-f

import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
AICs = {}
for k in range(1,len(features)):
    for variables in itertools.combinations(features, k):
        predictors = df_train[list(variables)]
        predictors['Intercept'] = 1
        res = sm.OLS(df_train.Temp, predictors).fit()
        AICs[variables] = 2*(k+1) - 2*res.llf
feature_after_ACI = list(pd.Series(AICs).idxmin())

/home/ramon/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [217]:
clf.fit(df_train[feature_after_ACI],df_train.Temp)
clf.score(df_train[feature_after_ACI],df_train.Temp)

0.75084089622168459

In [219]:
clf.fit(df_test[feature_after_ACI],df_test.Temp)
clf.score(df_test[feature_after_ACI],df_test.Temp)

0.6178654870315079